# Using Natural Language Processing techniques to understand the sentiment of twitter data

In [2]:
# NLP libraries
import nltk
import pandas as pd
from nltk.corpus import stopwords
import yaml
import re
# Libraries for Sentiment calls (Google)
import os
import json
from google.cloud import language # Google
from classes.splitter_postagger_nltk import Splitter, POSTagger, DictionaryTagger
# os.chdir("C:/Users/luisian/Documents/twitter/")

# Reading in some historical tweet data

In [24]:
# Reading in past tweets
import datetime
from pandas.tseries.offsets import *

df_all = pd.read_csv("sydneyseige5.csv", encoding='iso-8859-1', sep=',', infer_datetime_format=True)

# dropping tweets with no text
df_all = df_all.dropna(subset = ['text'])
df_all = df_all.reset_index(drop=True)

# calculating the timestamp in AEST
df_all['timestamp'] = (df_all['date'] + " " + df_all['time'])
df_all['timestamp'] = pd.to_datetime(df_all['timestamp'], infer_datetime_format=True)
df_all['timestamp'] = df_all['timestamp'] + Hour(11)
df = df_all

In [25]:
df_all

,id,date,time,user,text,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,timestamp
0,5.45E+17,15/12/2014,23:59:55,<Taddeh,My condolences to the victims of the #sydneys...,NaN,NaN,NaN,NaN,NaN,2014-12-16 10:59:55
1,5.45E+17,15/12/2014,23:59:41,<Sherbear2702,Grieving for the families of those lost in #s...,NaN,NaN,NaN,NaN,NaN,2014-12-16 10:59:41
2,5.45E+17,15/12/2014,23:59:23,<MarlowLuke,We are a country united in our grief today fo...,NaN,NaN,NaN,NaN,NaN,2014-12-16 10:59:23
3,5.45E+17,15/12/2014,23:59:11,<Dariabee87,"Why does someone on bail have a gun? ""Tori Jo...",NaN,NaN,NaN,NaN,NaN,2014-12-16 10:59:11
4,5.45E+17,15/12/2014,23:58:15,<JemimaBlandLD,Aussies are rejecting islamophobia & calling ...,NaN,NaN,NaN,NaN,NaN,2014-12-16 10:58:15
5,5.45E+17,15/12/2014,23:58:13,<RKMac65,"Stop immortalising this lunatics picture, he ...",NaN,NaN,NaN,NaN,NaN,2014-12-16 10:58:13
6,5.45E+17,15/12/2014,23:57:51,<Andonella,@AndrewBoltsBlog You really are a despicable ...,NaN,NaN,NaN,NaN,NaN,2014-12-16 10:57:51
7,5.45E+17,15/12/2014,23:57:36,<DannyjClayton,Jeez people! Sometimes it's wise to keep your...,NaN,NaN,NaN,NaN,NaN,2014-12-16 10:57:36
8,5.45E+17,15/12/2014,23:57:08,<AshleyDawn0829,"Deepest prayers to the family's in Sydney, lo...",NaN,NaN,NaN,NaN,NaN,2014-12-16 10:57:08
9,5.45E+17,15/12/2014,23:56:27,<RoHetherington,RT @msalimkassam: Awesome response from the A...,NaN,NaN,NaN,NaN,NaN,2014-12-16 10:56:27


# Preparing the data for Sentiment analysis and other AI techniques

This step involves: 

1) Cleaning out non-english characters

2) Cleaning up punctuation and standardizing text

3) Spell checking

4) Adding stop words -- i.e. words to omit because they add noise 

5) Lemmatization & Stemming (trying to reduce words such as "running, run, ran" to it's base word "run"

6) Other techniques


In [4]:
# Cleaning up the tweet data
tweets = []
stopwords_set = set(stopwords.words("english"))
counterillridewithyou = 1
countersydney = 1

# Going through all tweets to remove punctuation, lowercase, remove http and pics, remove stopwords
for item in df['text']:
    item = re.sub(r'[^\w\s\@]','',item)
    words_filtered = [e.lower() for e in item.split() if len(e) >= 2]
    words_cleaned = [word for word in words_filtered
        if 'http' not in word
        and 'pictwitter' not in word
        and '/' not in word
        and not word.startswith('@')
        and word != 'RT']
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]
    for word in words_cleaned:
        if word == 'illridewithyou':
            counterillridewithyou += 1
        if word == 'sydneyseige':
            countersydney += 1
    tweets.append(words_without_stopwords)

sentences=[]
for item in tweets:
    str1 = ' '.join(item)
    str1.encode("utf-8")
    sentences.append(str1)
# write post and pre
print("Original text: " + df['text'][1])
print("\n")
print("Cleaned text: " + sentences[1])


Original text:  Grieving for the families of those lost in #sydneyseige. Such a waste when lives are taken, no matter the reason or who is responsible.


Cleaned text: grieving families lost sydneyseige waste lives taken matter reason responsible


In [5]:
print("Quick stats about the tweets")
print("Number of tweets:", len(df['text']))
print("Number of I'll ride with you:", counterillridewithyou)
print("Number of SydneySeige:", countersydney)

Quick stats about the tweets
Number of tweets: 9624
Number of I'll ride with you: 866
Number of SydneySeige: 9421


In [6]:
print("Sample of the tweets: ")
sentences = sentences[1:100]
print(sentences[1:10])
df = df[1:100][:]

Sample of the tweets: 
['country united grief today innocent people lost lives sydneyseige prayers families', 'someone bail gun tori johnson reportedly wrestling gun gunman killed sydneyseige', 'aussies rejecting islamophobia calling better mental health care inspiration countries sydneyseige illridewithyou', 'stop immortalising lunatics picture doesnt deserve anything grief hes caused sydneyseige', 'really despicable human using sydneyseige vilify refugees', 'jeez people sometimes wise keep opinions im going take advicenot tweet anymore today sydneyseige', 'deepest prayers familys sydney love sent canada illridewithyou sydneyseige', 'rt awesome response australian public light sydneyseige illridewithyou', 'rip tori johnson katrina dawson sydneyseige prayforsydney']


# Understanding Sentiment based on specialised vocabulary or dictionaries

This is really powerful as it allows you to build your own dictionary of positive or negative phrases or words, which can be used add more contextual meaning. 

In the example below we are using a prebuilt dictionary of positive and negative words. 

In [9]:
# Generating a basis sentiment score 
from classes.splitter_postagger_nltk import Splitter, POSTagger, DictionaryTagger
pos=[]
for text in sentences:
    splitter = Splitter()
    postagger = POSTagger()
    splitted_sentences = splitter.split(text)
    pos_tagged_sentences = postagger.pos_tag(splitted_sentences)
    dicttagger = DictionaryTagger(['dict/positive.yml', 'dict/negative.yml', 'dict/inc.yml', 'dict/dec.yml', 'dict/inv.yml'])
    dict_tagged_sentences = dicttagger.tag(pos_tagged_sentences)
    pos.append(dict_tagged_sentences)

### Splitting sentences into words and tagging them with positive or negative connotations. This allows you to customize with your own unique SME vocabulary.

In [10]:
df['tagged'] = pd.Series(pos)
    
def value_of(sentiment):
    if sentiment == 'positive': return 1
    if sentiment == 'negative': return -1
    return 0

def sentence_score(sentence_tokens, previous_token, acum_score):    
    if not sentence_tokens:
        return acum_score
    else:
        current_token = sentence_tokens[0]
#         print ("current_token", current_token)
        tags = current_token[2]
#         print ("tags", tags)
        token_score = sum([value_of(tag) for tag in tags])
        if previous_token is not None:
            previous_tags = previous_token[2]
#             print ("previous_tags", previous_tags)
            if 'inc' in previous_tags:
                token_score *= 2.0
            elif 'dec' in previous_tags:
                token_score /= 2.0
            elif 'inv' in previous_tags:
                token_score *= -1.0
        return sentence_score(sentence_tokens[1:], current_token, acum_score + token_score)

def sentiment_score(review):
    return sum([sentence_score(sentence, None, 0.0) for sentence in review])
    
score = []    
for dict_tagged_sentences in df['tagged']:
    try: 
        s3 = sentiment_score(dict_tagged_sentences)
        score.append(s3)
    except:
        score.append(0)

df['score'] = pd.Series(score)


In [11]:
## add in a few negative words (curated top words)
df[['id', 'date', 'time', 'score', 'user', 'text', 'timestamp', 'tagged']]

,id,date,time,score,user,text,timestamp,tagged
1,5.45E+17,15/12/2014,23:59:41,0.0,<Sherbear2702,Grieving for the families of those lost in #s...,2014-12-16 10:59:41,"[[(country, country, ['NN']), (united, united,..."
2,5.45E+17,15/12/2014,23:59:23,0.0,<MarlowLuke,We are a country united in our grief today fo...,2014-12-16 10:59:23,"[[(someone, someone, ['NN']), (bail, bail, ['N..."
3,5.45E+17,15/12/2014,23:59:11,0.0,<Dariabee87,"Why does someone on bail have a gun? ""Tori Jo...",2014-12-16 10:59:11,"[[(aussies, aussies, ['NNS']), (rejecting, rej..."
4,5.45E+17,15/12/2014,23:58:15,0.0,<JemimaBlandLD,Aussies are rejecting islamophobia & calling ...,2014-12-16 10:58:15,"[[(stop, stop, ['VB']), (immortalising, immort..."
5,5.45E+17,15/12/2014,23:58:13,0.0,<RKMac65,"Stop immortalising this lunatics picture, he ...",2014-12-16 10:58:13,"[[(really, really, ['RB']), (despicable, despi..."
6,5.45E+17,15/12/2014,23:57:51,0.0,<Andonella,@AndrewBoltsBlog You really are a despicable ...,2014-12-16 10:57:51,"[[(jeez, jeez, ['NN']), (people, people, ['NNS..."
7,5.45E+17,15/12/2014,23:57:36,1.0,<DannyjClayton,Jeez people! Sometimes it's wise to keep your...,2014-12-16 10:57:36,"[[(deepest, deepest, ['JJS']), (prayers, praye..."
8,5.45E+17,15/12/2014,23:57:08,0.0,<AshleyDawn0829,"Deepest prayers to the family's in Sydney, lo...",2014-12-16 10:57:08,"[[(rt, rt, ['JJ']), (awesome, awesome, ['posit..."
9,5.45E+17,15/12/2014,23:56:27,0.0,<RoHetherington,RT @msalimkassam: Awesome response from the A...,2014-12-16 10:56:27,"[[(rip, rip, ['NN']), (tori, tori, ['NN']), (j..."
10,5.45E+17,15/12/2014,23:56:13,0.0,<bree_81,"@AbbeyRobinson15 ""@abbeyrobinson15: R.I.P Tor...",2014-12-16 10:56:13,"[[(sad, sad, ['JJ']), (lose, lose, ['NN']), (l..."


# Using external sentiment APIs. 

Whilst there is the option of building your own dictionary and oncology for sentiment, there is also the potential to use state of the art, sentiment APIs that we can leverage. 

Below is an example using Google sentiment API.

### What the Google sentiment means 

Score: 
Positive emotion score (greater than 0 to +1): 1 means more positive, 0 means netural. 
Negative emotion score (less than 0 to -1): -1 means more negative, 0 means netural. 
Neutral score (0): Can consist of mixed emotions, hence cannot decide if the sentiment is overall positive or negative.

Magnitude: 
Amount of emotional content. The higher the value the more emotion is expressed within the message, whereas a truly netural document (i.e describing facts) will have a low magnitude value. 
(i.e. "I love this, I hate this" will approximately give score (0), high magnitude. )

## What does Google sentiment think about Sydney Seige?

Using the Google APIs to extract out positive or negative sentiment

In [12]:
df = df.reset_index(drop=True)

# Imports the Google Cloud client library
#NB Google api does not distinguish between emotions ie sad or angy, just that they are negative or positive
# https://cloud.google.com/natural-language/docs/basics#sentiment-analysis-values
import os
import time
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
#os.chdir("C:/Users/luisian/Documents/twitter/Ross/")
credentials='MWHealth1-684bfd4453c8.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials
client = language.LanguageServiceClient()

# The text to analyze
score = []
mag=[]
t0 = time.time()

for line in range(10):#range(len(sentences)):
    text = sentences[line]
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects the sentiment of the text
    try:
        sentiment = client.analyze_sentiment(document=document).document_sentiment
        score.append(sentiment.score)
        mag.append(sentiment.magnitude)
    except:
        score.append('')
        mag.append('')
    
    if(line % 100 == 0):
        t1 = time.time()
        total_n = t1-t0
        
print("Google analysis on: " + sentences[1])
print("Sentiment score: " + str(score[0]))
print("Magnitude: " + str(mag[0]))

Google analysis on: country united grief today innocent people lost lives sydneyseige prayers families
Sentiment score: -0.800000011920929
Magnitude: 0.800000011920929


In [24]:
# Storing all the results in the same dataframe
df['Google_score'] = pd.Series(score)
df['Google_mag'] = pd.Series(mag)

dfprint = df[['Google_score', "Google_mag", "timestamp", "id", 'date', 'time', 'user', 'text']]
df[['text','Google_score', 'Google_mag']].head()

# Write Google sentiment to API
# dfprint.to_csv('googleAPI_sentiment_all.csv', sep='|', encoding='utf-8')

,text,Google_score,Google_mag
0,My condolences to the victims of the #sydneys...,-0.8,0.8
1,Grieving for the families of those lost in #s...,-0.1,0.1
2,We are a country united in our grief today fo...,0.0,0.0
3,"Why does someone on bail have a gun? ""Tori Jo...",0.6,0.6
4,Aussies are rejecting islamophobia & calling ...,-0.9,0.9


# Visual analytics that can empower you to unlock trends

In [17]:
# plotting libraries
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Layout, Scatter
from plotly import __version__
init_notebook_mode(connected=True)

In [18]:
!dir

 Volume in drive C is OSDisk
 Volume Serial Number is AE5E-DD86

 Directory of C:\Users\ashmaro1\Documents\_Projects\NSW_Health\sentiment

27/02/2018  08:54 AM    <DIR>          .
27/02/2018  08:54 AM    <DIR>          ..
27/02/2018  08:40 AM    <DIR>          .ipynb_checkpoints
27/02/2018  08:50 AM    <DIR>          classes
27/02/2018  08:40 AM    <DIR>          dict
27/02/2018  08:40 AM         1,741,516 googleAPI_sentiment_all_use.csv
27/02/2018  08:40 AM             2,389 MWHealth1-684bfd4453c8.json
27/02/2018  08:54 AM           148,701 Sentiment Analysis - SydneySeige.ipynb
27/02/2018  08:40 AM         1,539,161 sydneyseige5.csv
               4 File(s)      3,431,767 bytes
               5 Dir(s)  13,289,357,312 bytes free


In [27]:
df_all = pd.read_csv('googleAPI_sentiment_all_use.txt', sep='\t')
df_all['timestamp'] = pd.to_datetime(df_all.timestamp) + Hour(3)
# print((df2[['timestamp2', 'timestamp']]))
df_all.set_index(df_all.timestamp, inplace=True)

In [28]:
df_all

,Unnamed: 0,Unnamed: 0.1,id,date,time,timezone,user,text,tag,tweets,...,clean,Unnamed: 14,Google_score,Google_mag,timestamp,id.1,date.1,time.1,user.1,text.1
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-12-16 09:59:00,0,0,5.450000e+17,15/12/2014,23:59:55,AUS Eastern Standard Time,Taddeh,My condolences to the victims of the #sydneyse...,"#sydneyseige,,,,,,,,,,,","[my, condolences, to, the, victims, of, the, b...",...,my condolences to the victims of the but if yo...,0.0,-0.5,0.5,2014-12-16 09:59:00,5.450000e+17,15/12/2014,23:59:55,<Taddeh,My condolences to the victims of the #sydneys...
2014-12-16 09:59:00,1,1,5.450000e+17,15/12/2014,23:59:41,AUS Eastern Standard Time,Sherbear2702,Grieving for the families of those lost in #sy...,"#sydneyseige,,,,,,,,,,","[grieving, for, the, families, of, those, lost...",...,grieving for the families of those lost in suc...,1.0,-0.8,0.8,2014-12-16 09:59:00,5.450000e+17,15/12/2014,23:59:41,<Sherbear2702,Grieving for the families of those lost in #s...
2014-12-16 09:59:00,2,2,5.450000e+17,15/12/2014,23:59:23,AUS Eastern Standard Time,MarlowLuke,We are a country united in our grief today for...,"#SydneySeige,,,,,,,,,,,","[we, are, country, united, in, our, grief, tod...",...,we are country united in our grief today for t...,2.0,-0.1,0.1,2014-12-16 09:59:00,5.450000e+17,15/12/2014,23:59:23,<MarlowLuke,We are a country united in our grief today fo...
2014-12-16 09:59:00,3,3,5.450000e+17,15/12/2014,23:59:11,AUS Eastern Standard Time,Dariabee87,"""""Why does someone on bail have a gun? """"""""Tor...","#Ox7MEGvCrFOauEOE"",#SydneySeige,,,,,,,,","[""""why, does, someone, on, bail, have, gun?, ""...",...,why does someone on bail have gun tori johnson...,3.0,0.0,0.0,2014-12-16 09:59:00,5.450000e+17,15/12/2014,23:59:11,<Dariabee87,"Why does someone on bail have a gun? ""Tori Jo..."
2014-12-16 09:58:00,4,4,5.450000e+17,15/12/2014,23:58:15,AUS Eastern Standard Time,JemimaBlandLD,Aussies are rejecting islamophobia & calling f...,"#sydneyseige,#illridewithyou,,,,,,,,,,","[aussies, are, rejecting, islamophobia, callin...",...,aussies are rejecting islamophobia calling for...,4.0,0.6,0.6,2014-12-16 09:58:00,5.450000e+17,15/12/2014,23:58:15,<JemimaBlandLD,Aussies are rejecting islamophobia & calling ...
2014-12-16 09:58:00,5,5,5.450000e+17,15/12/2014,23:58:13,AUS Eastern Standard Time,RKMac65,"Stop immortalising this lunatics picture, he d...","#SydneySeige,,,,,,,,,,","[stop, immortalising, this, lunatics, picture,...",...,stop immortalising this lunatics picture he do...,5.0,-0.9,0.9,2014-12-16 09:58:00,5.450000e+17,15/12/2014,23:58:13,<RKMac65,Stop immortalising this lunatics picture
2014-12-16 09:57:00,6,6,5.450000e+17,15/12/2014,23:57:51,AUS Eastern Standard Time,Andonella,@AndrewBoltsBlog You really are a despicable h...,"#sydneyseige,,,,,,,,,,","[you, really, are, despicable, human, being,, ...",...,you really are despicable human being using to...,6.0,-0.3,0.3,2014-12-16 09:57:00,5.450000e+17,15/12/2014,23:57:51,<Andonella,@AndrewBoltsBlog You really are a despicable ...
2014-12-16 09:57:00,7,7,5.450000e+17,15/12/2014,23:57:36,AUS Eastern Standard Time,DannyjClayton,Jeez people! Sometimes it's wise to keep your ...,"#sydneyseige,,,,,,,,,,,","[jeez, people!, sometimes, it's, wise, to, kee...",...,jeez people sometimes its wise to keep your op...,7.0,0.1,0.1,2014-12-16 09:57:00,5.450000e+17,15/12/2014,23:57:36,<DannyjClayton,Jeez people! Sometimes it's wise to keep your...
2014-12-16 09:57:00,8,8,5.450000e+17,15/12/2014,23:57:08,AUS Eastern Standard Time,AshleyDawn0829,"Deepest prayers to the family's in Sydney, lov...","#illridewithyou,#Sydneyseige,,,,,,,,,","[deepest, prayers, to, the, family's, in, sydn...",...,deepest prayers to the familys in sydney love ...,8.0,0.7,0.7,2014-12-16 09:57:00,5.450000e+17,15/12/2014,23:57:08,<AshleyDawn0829,Deepest prayers to the family's in Sydney


In [38]:
# resample to every hour
df_hourly = df_all.resample('H').count()

In [39]:
df_hourly

,Unnamed: 0,Unnamed: 0.1,id,date,time,timezone,user,text,tag,tweets,...,clean,Unnamed: 14,Google_score,Google_mag,timestamp,id.1,date.1,time.1,user.1,text.1
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-12-15 10:00:00,329,329,329,329,329,329,329,329,329,329,...,322,329,329,329,329,329,329,329,329,329
2014-12-15 11:00:00,589,589,589,589,589,589,589,589,588,589,...,581,589,588,588,589,589,589,589,589,589
2014-12-15 12:00:00,487,487,487,487,487,487,487,487,487,487,...,479,487,483,483,487,487,487,487,487,487
2014-12-15 13:00:00,392,392,392,392,392,392,392,392,392,392,...,386,392,392,392,392,392,392,392,392,392
2014-12-15 14:00:00,467,467,467,467,467,467,467,467,467,467,...,459,467,467,467,467,467,467,467,467,467
2014-12-15 15:00:00,469,469,469,469,469,469,469,469,469,469,...,460,469,468,468,469,469,469,469,469,469
2014-12-15 16:00:00,412,412,412,412,412,412,412,412,411,412,...,409,412,410,410,412,412,412,412,412,412
2014-12-15 17:00:00,393,393,393,393,393,393,393,393,393,393,...,388,393,393,393,393,393,393,393,393,393
2014-12-15 18:00:00,377,377,377,377,377,377,377,377,377,377,...,369,377,373,373,377,377,377,377,377,377


In [41]:
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True) 

# Volume of tweets over time
data = [Scatter(x = df_hourly.index, y = df_hourly.id, mode = 'marker', marker = dict(color = 'rgb(170,204,235)'))]
layout = Layout(title="Volume of tweets over time", 
                yaxis = layoutaxis,
                font = dict(family='Courier New, monospace', size=18, color='rgb(0,0,0)')
               )

layoutaxis = dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        autotick=True,
        autorange=True,
        showticklabels=True,
        title = 'Volume'
    )
fig= Figure(data=data, layout=layout)

# Plot and embed in ipython notebook!
iplot(fig, filename='basic-scatter')




In [36]:
# calculating the mean value for hourly and minute
df_hourly_avg = df_all.resample('H').mean()
df_minute_avg = df_all.resample('T').mean()

In [42]:
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np


# Create a trace
trace = go.Scatter(
    x = df_hourly_avg['Google_score'].tolist(),
    y = df_hourly_avg['Google_mag'].tolist(),
    mode = 'markers'
)

data = [trace]

layout= go.Layout(
    title= 'Sydney seige tweet sentiment - more negative than positive',
    hovermode= 'closest',
    xaxis= dict(
        title= 'Sentiment score',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Magnitude score',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend= False
)
fig= go.Figure(data=data, layout=layout)

# Plot and embed in ipython notebook!
iplot(fig, filename='basic-scatter')


In [33]:
## tweet sentiment over time
data = Scatter(x = df_hourly_avg.index, y = df_hourly_avg.Google_score, marker = dict(color = 'rgb(170,204,235)',))
# py.iplot(data)
layoutaxis = dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        autotick=True,
        autorange=True,
        showticklabels=True,
        title = 'Google Sentiment Score'
    )
iplot({

                         "data": [

                                  data

                                  ], 

                        "layout": Layout(title="Sentiment of tweets over time", yaxis = layoutaxis,font=dict(family='Courier New, monospace', size=18, color='rgb(0,0,0)'))

                                  },filename='Line_chart.html',image='jpeg')



In [43]:
## tweet Ross sentiment over time
data = Scatter(x = df_hourly_avg.index, y = df_hourly_avg.Google_mag, marker = dict(color = 'rgb(170,204,235)',))
# py.iplot(data)
layoutaxis = dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        autotick=True,
        autorange=True,
        showticklabels=True,
        title = 'Google Sentiment Magnitude'
    )
iplot({

                         "data": [

                                  data

                                  ], 

                        "layout": Layout(title="Sentiment of tweets over time", yaxis = layoutaxis,font=dict(family='Courier New, monospace', size=18, color='rgb(0,0,0)'))

                                  },filename='Line_chart.html',image='jpeg')

